In [ ]:
from pathlib import Path

from IPython.display import Image
from lsst.daf.butler import Butler

In [ ]:
DATASTORE = '/work/datastore/'
COLLECTIONS = 'PFS/default'
INSTRUMENT = 'lsst.obs.pfs.PrimeFocusSpectrograph'
NUM_JOBS = 12

DRP_DIR = !echo $DRP_STELLA_DIR
PIPELINE_PATH = f'{DRP_DIR[0]}/pipelines/reduceExposure.yaml'
QA_PIPELINE_PATH = f'/work/wtg/drp_qa_dev/pipelines/detectorMapQa.yaml'

OUTPUT_COLLECTION = 'wtg/PIPE2D-1528'
LOG_FILE = Path('./pipetask.log').absolute().as_posix()

In [ ]:
!defineCombination.py --update {DATASTORE} PFS wtg/run18_dm_inputs \
    113981 113982 113983 113984 113985 113986 113987 113988 113989 113990 \
    113991 113992 113993 113994 113995 114641 114642 114644 114645 114646 \
    114647 114648 114649 114650 114651 114652 114653 114654 114655 114656 \
    114657 114659 114660 114661 114986 114987 114988 114989 114990 114991 \
    114992 114993 114994 114995 114996 114997 114999 114500

## Run `reduceExposure` pipeline

In [ ]:
!pipetask \
    --long-log \
    --no-log-tty \
    --log-level "pfs.drp.stella=INFO" \
    --log-file {LOG_FILE} \
    run \
    -b {DATASTORE} \
    -j 50 \
    --instrument {INSTRUMENT} \
    -i {COLLECTIONS} \
    -o {OUTPUT_COLLECTION} \
    -p {PIPELINE_PATH} \
    -d "combination in ('wtg/run18_dm_origin_inputs') AND arm in ('b', 'r', 'n')"

## Run `detectorMapQa` pipeline

In [ ]:
make_residual = True
combine = False
use_sigma = False
spatial_range = 0.1
wavelength_range = 0.1

In [ ]:
!pipetask \
    --log-tty \
    --long-log \
    --log-level "pfs.detectorMapCombinedQa=INFO" \
    --log-file {LOG_FILE} \
    run \
    --register-dataset-types \
    --replace-run \
    -b {DATASTORE} \
    -j 50 \
    --instrument {INSTRUMENT} \
    -i {COLLECTIONS} \
    -o {OUTPUT_COLLECTION} \
    -p "{QA_PIPELINE_PATH}" \
    -d "combination = 'wtg/run18_dm_origin_inputs'" \
    -c dmResiduals:useSigmaRange={use_sigma} \
    -c dmResiduals:spatialRange={spatial_range} \
    -c dmResiduals:wavelengthRange={wavelength_range} \
    --fail-fast

## Get stored objects from the butler

In [ ]:
butler = Butler(DATASTORE, collections=OUTPUT_COLLECTION)

In [ ]:
butler.get('dmQaDetectorStats')

In [ ]:
butler.getURI('dmQaCombinedResidualPlot')

In [ ]:
!cp {butler.getURI('dmQaCombinedResidualPlot').unquoted_path} .

In [ ]:
data_id = dict(arm='r', spectrograph=2, exposure=113595)

In [ ]:
butler.get('dmQaResidualStats', data_id).T

In [ ]:
Image(filename=butler.getURI('dmQaResidualPlot', data_id).unquoted_path)